In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

开始使用Vertex AI数据标记

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/datasets/get_started_with_data_labeling.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/datasets/get_started_with_data_labeling.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/datasets/get_started_with_data_labeling.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI工作台中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概览

本教程演示了如何在生产中使用 Vertex AI。本教程涵盖了数据管理：如何开始使用 Vertex AI 数据标记服务。

了解更多关于[Vertex AI 数据标记](https://cloud.google.com/vertex-ai/docs/datasets/data-labeling-job)。

### 目标

在本教程中，您将学习如何使用`Vertex AI数据标记`服务。

本教程使用以下Google Cloud ML服务：

- `Vertex AI数据标记`
- `Vertex AI数据集`

执行的步骤包括：

- 为数据标记员创建专家池。
- 创建一个数据标记作业。
- 提交数据标记作业。
- 列出数据标记作业。
- 取消一个数据标记作业。

了解更多关于[请求Vertex AI数据标记作业](https://cloud.google.com/vertex-ai/docs/datasets/data-labeling-job)。

数据集

本教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[花卉数据集](https://www.tensorflow.org/datasets/catalog/tf_flowers)。您在本教程中使用的数据集版本存储在一个公共云存储桶中。训练好的模型可以预测一幅图像属于五种花中的哪一种：雏菊、蒲公英、玫瑰、向日葵或郁金香。

### 成本

本教程使用了 Google Cloud 的计费组件：
* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和[Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [价格计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成成本估算。

## 安装

安装以下包以执行此笔记本。

In [ ]:
import os

# Install the packages
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage 

if os.getenv("IS_TESTING"):
    ! pip3 install --upgrade --quiet google-api-core==2.10 

只有Colab：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

在开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下步骤：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
*查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

邮件

您需要一个电子邮件地址来发送标注工作请求。这个电子邮件地址将成为数据标注专家团队的经理。

在本教程中，如果您没有指定电子邮件地址，将使用与您的项目ID关联的电子邮件地址。

In [ ]:
EMAIL = "[your-email-address]"  # @param {type: "string"}

if os.getenv("IS_TESTING"):
    EMAIL = "noreply@google.com"

In [ ]:
if EMAIL == "[your-email-address]":
    shell_output = ! gcloud auth list 2>/dev/null
    EMAIL = shell_output[2].replace("*", "").strip()

print(EMAIL)

###验证您的Google云账户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关说明进行操作。

**1. Vertex AI工作台**
* 无需操作，因为您已经通过验证。

**2. 本地JupyterLab实例，取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务帐户或其他
* 请参考如何授予您的服务帐户云存储权限，网址为https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples。

创建一个云存储桶

创建一个存储桶来存储中间产物，比如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

导入Vertex AI SDK

将Vertex AI SDK导入到我们的Python环境中。

In [ ]:
import os
import time

import google.cloud.aiplatform as aip
from google.cloud import storage
from google.cloud.aiplatform import gapic
from google.protobuf.json_format import ParseDict
from google.protobuf.struct_pb2 import Value

### 初始化 Python 的 Vertex AI SDK

为您的项目和对应的存储桶初始化 Python 的 Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, location=REGION)

#### Vertex AI 常量

为 Vertex AI 设置以下常量：

- `API_ENDPOINT`：用于数据集、模型、作业、流水线和终端服务的 Vertex AI API 服务端点。
- `PARENT`：数据集、模型和终端资源的 Vertex AI 位置根路径。

In [ ]:
# API Endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex AI location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

模式常量

接下来，为与图像分类数据集相关的模式设置常量：

- 数据标注（注释）模式：告诉受管理的数据集服务数据如何被标记（注释）。

In [ ]:
# Image labeling task
LABELING_SCHEMA_IMAGE = "gs://google-cloud-aiplatform/schema/datalabelingjob/inputs/image_classification_1.0.0.yaml"

## 创建客户端

Vertex AI SDK 采用客户端/服务器模型。在您的一侧（Python 脚本）创建一个客户端，用来向服务器（Vertex AI）发送请求并接收响应。

在本教程中，您会使用多个客户端，因此请提前设置好它们。

- 专业池服务的专业池客户端
- 数据标记的作业服务客户端

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}

clients = {}
clients["job"] = gapic.JobServiceClient(client_options=client_options)

# add client for specialist pool
clients["specialist_pool"] = gapic.SpecialistPoolServiceClient(
    client_options=client_options
)

for client in clients.items():
    print(client)

### 为要标记的示例创建一个CSV文件

接下来，您将为要求进行标记的示例创建一个CSV文件。

在这个示例中，要标记的示例是图像。在CSV文件的每一行中，您需要指定图像的Cloud Storage位置。

In [ ]:
test_filename = "labeling.csv"
LABELING_FILES = [
    "gs://cloud-samples-data/vision/automl_classification/flowers/daisy/100080576_f52e8ee070_n.jpg",
    "gs://cloud-samples-data/vision/automl_classification/flowers/daisy/102841525_bd6628ae3c.jpg",
]

IMPORT_FILE = BUCKET_URI + "/labeling.csv"

bucket = storage.Client(project=PROJECT_ID).bucket(BUCKET_URI.replace("gs://", ""))

# creating a blob
blob = bucket.blob(blob_name=test_filename)

# creating data variable
data = LABELING_FILES[0] + "\n" + LABELING_FILES[1] + "\n"

# uploading data variable content to bucket
blob.upload_from_string(data, content_type="text/csv")

# printing path of uploaded file
print(IMPORT_FILE)

# printing content of uploaded file
! gsutil cat $IMPORT_FILE

## 创建一个无标签的数据集

接下来，您可以为待标记的数据创建一个数据集。

In [ ]:
dataset = aip.ImageDataset.create("labeling")
print(dataset)

## 导入未标记的数据

现在，将未标记的数据导入数据集，即待标记的示例。

In [ ]:
dataset.import_data(
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aip.schema.dataset.ioformat.image.single_label_classification,
)

## 创建一个新的数据专家池

您的数据标注工作将被发送到一个数据专家池中。您可以拥有一个或多个专家池。

在下一步中，您可以使用`create_specialist_pool()`方法创建一个新的专家池。请求包括以下参数：

- `name`：专家池的资源名称。
- `display_name`：专家池的人类可读名称。
- `specialist_manager_emails`：专家池经理的电子邮件地址列表。

*注意：* 如果已经存在一个专家池，您可以使用现有的专家池。

In [ ]:
specialist_pool = {
    "name": "labeling",
    "display_name": "labeling",
    "specialist_manager_emails": [EMAIL],
}

request = clients["specialist_pool"].create_specialist_pool(
    parent=PARENT, specialist_pool=specialist_pool
)

result = request.result()
print(result)

specialist_name = result.name

specialist_id = specialist_name.split("/")[-1]

print(specialist_name)

创建数据标记工作

现在您有了专家池，您可以使用`create_data_labeling_job()`方法发送数据标记请求。

您的请求将包括以下内容：

- 具有未标记数据的Vertex AI数据集。
- 标记说明。

In [ ]:
# create placeholder file for instructions for data labeling
! echo "this is instruction" >> instruction.txt | gsutil cp instruction.txt $BUCKET_URI

In [ ]:
LABLEING_SCHEMA = LABELING_SCHEMA_IMAGE
INSTRUCTION_FILE = BUCKET_URI + "/instruction.txt"

inputs = ParseDict({"annotation_specs": ["rose"]}, Value())

data_labeling_job = {
    "display_name": "labeling",
    "datasets": [dataset.resource_name],
    "labeler_count": 1,
    "instruction_uri": INSTRUCTION_FILE,
    "inputs_schema_uri": LABLEING_SCHEMA,
    "inputs": inputs,
    "annotation_labels": {
        "aiplatform.googleapis.com/annotation_set_name": "data_labeling_job_specialist_pool"
    },
    "specialist_pools": [specialist_name],
}

print(data_labeling_job)

request = clients["job"].create_data_labeling_job(
    parent=PARENT, data_labeling_job=data_labeling_job
)

print(request)

labeling_task_name = request.name

print(labeling_task_name)

### 获取数据标注工作

您可以使用`get_data_labeling_job()`方法获取有关数据标注工作的信息，需要提供以下参数：

- `name`：标注任务的名称。

In [ ]:
request = clients["job"].get_data_labeling_job(name=labeling_task_name)
print(request)

### 取消数据标注任务

您可以使用`cancel_data_labeling_job()`方法取消数据标注请求，其中包含以下参数:

- `name`: 标注任务的名称。

In [ ]:
request = clients["job"].cancel_data_labeling_job(name=labeling_task_name)
print(request)

等待标记工作被取消

取消请求是异步的。下面的代码会轮询标记工作的状态，直到状态为CANCELED。

In [ ]:
while True:
    response = clients["job"].get_data_labeling_job(name=labeling_task_name)
    if response.state == gapic.JobState.JOB_STATE_CANCELLED:
        print("Labeling job CANCELED")
        break
    else:
        print("Canceling labeling job:", response.state)
        time.sleep(60)

清理

要清理此项目中使用的所有GCP资源，您可以[删除用于教程的GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
# Set this to true only if you'd like to delete your bucket
delete_bucket = False

# Delete the dataset using the Vertex AI fully qualified identifier for the dataset
dataset.delete()

# Delete the labeling job using the Vertex AI fully qualified identifier for the dataset
request = clients["job"].delete_data_labeling_job(name=labeling_task_name)

# Delete the specialist pool using the Vertex AI fully qualified identifier for the dataset
clients["specialist_pool"].delete_specialist_pool(name=specialist_name)

# Delete the bucket created
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI